# 1. Install Dependencies

In [1]:
# Install required libraries
!pip install -U datasets transformers evaluate peft
!apt-get install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 2. Preprocess Data

In [2]:
# Load data
from datasets import load_dataset
imdb = load_dataset("imdb")
print(imdb)

train_dataset = imdb['train'].shuffle(seed=42).select(range(500))
test_dataset = imdb['test'].shuffle(seed=42).select(range(500))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [3]:
from datasets import load_dataset
import torch
import transformers
from transformers import Trainer, TrainingArguments
from transformers.models.auto import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
from transformers import DataCollatorWithPadding
import evaluate

transformers.set_seed(42)
torch.manual_seed(42)

CHECKPOINT = "microsoft/deberta-v3-base"
save_path = "./deberta-v3"

tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)




tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [4]:
config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

static_memory = model.get_memory_footprint()/ (2 ** 20)
print(f"Static memory footprint: {static_memory:.2f} MB")


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Static memory footprint: 703.52 MB


In [5]:
best_lr = 2e-5
best_batch_size = 32
best_epoch = 2

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}


training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Reset peak tracking
torch.cuda.reset_peak_memory_stats()

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: huangyj2002 (huangyj2002-nanyang-technological-university-singapore) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=32, training_loss=0.6540226936340332, metrics={'train_runtime': 54.1637, 'train_samples_per_second': 18.463, 'train_steps_per_second': 0.591, 'total_flos': 263115773952000.0, 'train_loss': 0.6540226936340332, 'epoch': 2.0})


{'eval_loss': 0.5485714077949524, 'eval_accuracy': 0.85, 'eval_f1': 0.8251748251748252, 'eval_recall': 0.7195121951219512, 'eval_precision': 0.9672131147540983, 'eval_runtime': 7.2065, 'eval_samples_per_second': 69.382, 'eval_steps_per_second': 2.22, 'epoch': 2.0}


In [6]:
peak_memory = torch.cuda.max_memory_allocated() / (2 ** 20)
print(f"Peak memory usage: {peak_memory:.2f} MB")

Peak memory usage: 33708.59 MB


#3. LoRA (lr: 2e-5)

In [4]:
from peft import LoraConfig, get_peft_model

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)


lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_proj", "key_proj", "value_proj"],
    modules_to_save=["classifier", "pooler"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS",
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,034,498 || all params: 185,458,180 || trainable%: 0.5578


In [9]:
static_memory = model.get_memory_footprint()/ (2 ** 20)
print(f"Static memory footprint: {static_memory:.2f} MB")

Static memory footprint: 707.47 MB


In [5]:
from transformers import DataCollatorWithPadding

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

best_lr = 2e-5
best_batch_size = 32
best_epoch = 2

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Reset peak tracking
torch.cuda.reset_peak_memory_stats()

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: huangyj2002 (huangyj2002-nanyang-technological-university-singapore) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=32, training_loss=0.6930218935012817, metrics={'train_runtime': 37.6498, 'train_samples_per_second': 26.561, 'train_steps_per_second': 0.85, 'total_flos': 266293751808000.0, 'train_loss': 0.6930218935012817, 'epoch': 2.0})


{'eval_loss': 0.6932774782180786, 'eval_accuracy': 0.49, 'eval_f1': 0.6577181208053692, 'eval_recall': 0.9959349593495935, 'eval_precision': 0.4909819639278557, 'eval_runtime': 5.3361, 'eval_samples_per_second': 93.702, 'eval_steps_per_second': 2.998, 'epoch': 2.0}


In [6]:
peak_memory = torch.cuda.max_memory_allocated() / (2 ** 20)
print(f"Peak memory usage: {peak_memory:.2f} MB")

Peak memory usage: 31235.82 MB


#4. LoRA (lr: 1e-3)

In [8]:
from peft import LoraConfig, get_peft_model

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)


lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_proj", "key_proj", "value_proj"],
    modules_to_save=["classifier", "pooler"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS",
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,034,498 || all params: 185,458,180 || trainable%: 0.5578


In [9]:
from transformers import DataCollatorWithPadding

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

best_lr = 1e-3
best_batch_size = 32
best_epoch = 2

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

Step,Training Loss


TrainOutput(global_step=32, training_loss=0.6889163255691528, metrics={'train_runtime': 25.0588, 'train_samples_per_second': 39.906, 'train_steps_per_second': 1.277, 'total_flos': 266293751808000.0, 'train_loss': 0.6889163255691528, 'epoch': 2.0})


{'eval_loss': 0.571191668510437, 'eval_accuracy': 0.832, 'eval_f1': 0.8037383177570093, 'eval_recall': 0.6991869918699187, 'eval_precision': 0.945054945054945, 'eval_runtime': 5.2866, 'eval_samples_per_second': 94.579, 'eval_steps_per_second': 3.027, 'epoch': 2.0}


# 5. Prompt Tuning (lr: 2e-5)


In [10]:
from peft import PromptTuningConfig, get_peft_model

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)

prompt_config = PromptTuningConfig(
    task_type="SEQ_CLS",
    num_virtual_tokens=20,
    token_dim=768,
    num_transformer_submodules=1,
    num_attention_heads=12,
    num_layers=12,
    tokenizer_name_or_path=CHECKPOINT,
)

prompt_config.modules_to_save = ["classifier", "pooler"]

model = get_peft_model(model, prompt_config)
model.print_trainable_parameters()



Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 607,490 || all params: 185,031,172 || trainable%: 0.3283


In [11]:
from transformers import DataCollatorWithPadding

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

best_lr = 2e-5
best_batch_size = 32
best_epoch = 2

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Reset peak tracking
torch.cuda.reset_peak_memory_stats()

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

Step,Training Loss


TrainOutput(global_step=32, training_loss=0.7056373953819275, metrics={'train_runtime': 24.9785, 'train_samples_per_second': 40.035, 'train_steps_per_second': 1.281, 'total_flos': 264934797312000.0, 'train_loss': 0.7056373953819275, 'epoch': 2.0})


{'eval_loss': 0.6925653219223022, 'eval_accuracy': 0.512, 'eval_f1': 0.016129032258064516, 'eval_recall': 0.008130081300813009, 'eval_precision': 1.0, 'eval_runtime': 5.5013, 'eval_samples_per_second': 90.888, 'eval_steps_per_second': 2.908, 'epoch': 2.0}


In [12]:
peak_memory = torch.cuda.max_memory_allocated() / (2 ** 20)
print(f"Peak memory usage: {peak_memory:.2f} MB")

Peak memory usage: 30065.86 MB


# 6. Prompt Tuning (lr: 1e-3)

In [13]:
from peft import PromptTuningConfig, get_peft_model

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)

prompt_config = PromptTuningConfig(
    task_type="SEQ_CLS",
    num_virtual_tokens=20,
    token_dim=768,
    num_transformer_submodules=1,
    num_attention_heads=12,
    num_layers=12,
    tokenizer_name_or_path=CHECKPOINT,
)

prompt_config.modules_to_save = ["classifier", "pooler"]

model = get_peft_model(model, prompt_config)
model.print_trainable_parameters()



Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 607,490 || all params: 185,031,172 || trainable%: 0.3283


In [14]:
from transformers import EarlyStoppingCallback
from transformers import DataCollatorWithPadding

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

early_stopping = EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.0)

batch_size = 32
lr = 1e-3
epoch = 15

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    load_best_model_at_end=True,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

# Reset peak tracking
torch.cuda.reset_peak_memory_stats()

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Recall,Precision
1,No log,0.703482,0.492000,0.659517,1.000000,0.492000
2,No log,0.740898,0.510000,0.008097,0.004065,1.000000
3,No log,0.819028,0.508000,0.000000,0.000000,0.000000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=48, training_loss=0.7692001660664877, metrics={'train_runtime': 53.7606, 'train_samples_per_second': 139.507, 'train_steps_per_second': 4.464, 'total_flos': 397402195968000.0, 'train_loss': 0.7692001660664877, 'epoch': 3.0})


{'eval_loss': 0.7034820914268494, 'eval_accuracy': 0.492, 'eval_f1': 0.6595174262734584, 'eval_recall': 1.0, 'eval_precision': 0.492, 'eval_runtime': 5.488, 'eval_samples_per_second': 91.108, 'eval_steps_per_second': 2.915, 'epoch': 3.0}
